In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from selenium import webdriver

In [20]:
# change path to chromedriver on your computer
PATH = "/home/gsunit/development/chromedriver"

# url to uber-eats page
# url = "https://www.ubereats.com/city/new-york-city"
url = "https://www.ubereats.com/city/new-york-city"

In [21]:
driver = webdriver.Chrome(PATH)
driver.get(url)
print("Successfully opened: ", driver.title)

Successfully opened:  Best New York City Restaurants | Food Delivery & Takeout


In [22]:
class_name = 'dn bk bl at fn al aj fo dl'
xpath_name = f"//h3[@class='{class_name}']/a"

search = driver.find_elements_by_xpath(xpath_name)
search

[<selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="e5d1f9a7-d953-4593-aba7-6c413f6a8c5a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="f66a4955-c5f9-4565-8b31-78d4a4890594")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="17f25654-43f1-4b64-8a98-13cb13988320")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="b0d29b22-bdc4-4403-b3e7-e823c6b107c4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="df443773-f5ae-44cf-967f-8f8d46d31199")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="5c705294-ecb2-4306-9f28-ad2614d363f8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f0e0dee77d03ff7e86bd2e554437ca50", element="8bd5a86b-527b-4c99-86b0-17